In [1]:
import PyPDF2
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import pandas as pd
from pytesseract import image_to_string
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract'
import os
import re

In [2]:
## Fetching PDF files

pdfs=[]
for file in os.listdir():
    if '.pdf' in file:
        pdfs.append(file)

In [91]:
# Getting certificate license code from PDF

def get_code(pdf):
    with open(pdf, 'rb') as file:
        fileReader = PyPDF2.PdfFileReader(file)
        if fileReader.isEncrypted:
            fileReader.decrypt('')
        page=fileReader.getPage(0)
        page_content = page.extractText().split()

    code=[i for i in page_content if '/verify' in i][0].split('/verify')[1]
    if 'Coursera' in code:
        code=code[:code.index('Coursera')]
    return code

In [4]:
def pdf_to_image(pdf):
    pages = convert_from_path(pdf, 500)
    for page in pages:
        page.save('out.png', 'PNG')
    image=Image.open('out.png')
    return image

In [345]:
# Fetching Date, person, Course Name, Organization, URL from the PDF

def course_details(image,code):
    text=image_to_string(image)
    date_index=-1
    for line in text.split('\n'):
        date_index+=1
        if '/' in line:
            break

    text_list=text.split('\n')[date_index:]
    text_list=[i for i in text_list if ('' != i)]
    text_list=[i for i in text_list if (' ' != i)]

    new_text=' '.join(text_list)

    date=text_list[0]
    print(date)

    link='https://www.'+(new_text.split('/verify')[0]).split()[-1]+'/account/accomplishments/certificate'+(new_text.split('/verify')[1]).split()[0]
    if code!=(new_text.split('/verify')[1]).split()[0]:
        other_link='https://www.'+(new_text.split('/verify')[0]).split()[-1]+'/account/accomplishments/certificate/'+code
    else:
        other_link=None
    print(link,other_link)

    course_pattern1=r'completed(.+)a \d week'
    course_pattern2=r'completed(.+)an online'
    course_pattern3=r'distinction(.+)a 4 week'
    course_pattern4=r'distinction(.+)an online'

    course_regex1=re.compile(course_pattern1)
    course_regex2=re.compile(course_pattern2)
    course_regex3=re.compile(course_pattern3)
    course_regex4=re.compile(course_pattern4)

    course_search1=course_regex1.search(new_text)
    course_search2=course_regex2.search(new_text)
    course_search3=course_regex3.search(new_text)
    course_search4=course_regex4.search(new_text)

    if course_search3:
        course=course_search3.groups()[0].strip()
    elif course_search4:
        course=course_search4.groups()[0].strip()
    elif course_search1:
        course=course_search1.groups()[0].strip()
    elif course_search2:
        course=course_search2.groups()[0].strip()
    else:
        course=None
    print(course)

    organization_pattern=r'authorized by(.+)and offered'
    organization_regex=re.compile(organization_pattern)
    organization_search=organization_regex.search(new_text)
    organization=organization_search.groups()[0].strip()
    print(organization)

    name=' '.join([i for i in text_list[1].split() if i.isupper()][:2])
    print(name)
    print()

    return [date,name,course,organization,link,other_link]

In [346]:
# Extracting information from all PDFs

info=[]
error_pdfs=[]
for pdf in pdfs:
    try:
        code=get_code(pdf)
        image=pdf_to_image(pdf)
        info.append(course_details(image,code)+[pdf])
    except:
        print(pdf)
        print()
        error_pdfs.append(pdf)\
        
        continue

10/11/2019
https://www.coursera.org/account/accomplishments/certificate/KLT83QMUTE3DQ https://www.coursera.org/account/accomplishments/certificate//KLT8QMUTE3DQ
AI Workflow: Data Analysis and Hypothesis Testing
IBM
SHAURYA KHURANA

10/15/2019
https://www.coursera.org/account/accomplishments/certificate/6WD5S5Q0JD3F8 https://www.coursera.org/account/accomplishments/certificate//6WD5S5QJD3F8
AI Workflow: Feature Engineering and Bias Detection
IBM
SHAURYA KHURANA

11/20/2018
https://www.coursera.org/account/accomplishments/certificate/5EAGTSLUCGHD None
AWS Fundamentals: Going Cloud-Native
Amazon Web Services
SHAURYA KHURANA

06/14/2015
https://www.coursera.org/account/accomplishments/certificate/PKU8LD3X8G25 None
Budgeting and Scheduling Projects
University of California, Irvine
SHAURYA KHURANA

10/21/2016
https://www.coursera.org/account/accomplishments/certificate/T2JF8FDHTZH4 None
Business Metrics for Data-Driven Companies
Duke University
SHAURYA KHURANA

09/18/2015
https://www.courser

In [348]:
error_pdfs

['Introduction to Project Management Specialization.pdf']

In [347]:
data=pd.DataFrame(info)
data.columns=['Date','Name','Course','Organization','Link','Alternate Link','Filename']
data['Date']=pd.to_datetime(data['Date'],format='%m/%d/%Y')
data

,Date,Name,Course,Organization,Link,Alternate Link,Filename
0,2019-10-11,SHAURYA KHURANA,AI Workflow: Data Analysis and Hypothesis Testing,IBM,https://www.coursera.org/account/accomplishmen...,https://www.coursera.org/account/accomplishmen...,AI Workflow- Data Analysis and Hypothesis.pdf
1,2019-10-15,SHAURYA KHURANA,AI Workflow: Feature Engineering and Bias Dete...,IBM,https://www.coursera.org/account/accomplishmen...,https://www.coursera.org/account/accomplishmen...,AI Workflow- Feature Engineering and Bias Dete...
2,2018-11-20,SHAURYA KHURANA,AWS Fundamentals: Going Cloud-Native,Amazon Web Services,https://www.coursera.org/account/accomplishmen...,None,AWS Fundamentals- Going Cloud Native.pdf
3,2015-06-14,SHAURYA KHURANA,Budgeting and Scheduling Projects,"University of California, Irvine",https://www.coursera.org/account/accomplishmen...,None,Budgeting and Scheduling Projects.pdf
4,2016-10-21,SHAURYA KHURANA,Business Metrics for Data-Driven Companies,Duke University,https://www.coursera.org/account/accomplishmen...,None,Business Metrics for Data-Driven Companies.pdf
5,2015-09-18,SHAURYA KHURANA,Corporate Financial Decision-Making for Value ...,The University of Melbourne,https://www.coursera.org/account/accomplishmen...,None,Corporate Financial Decision-Making for Value ...
6,2019-10-06,SHAURYA KHURANA,Project: Multiple Linear Regression with sciki...,Rhyme,https://www.coursera.org/account/accomplishmen...,https://www.coursera.org/account/accomplishmen...,Coursera Multiple Regression Machine Learning.pdf
7,2018-05-30,SHAURYA KHURANA,Data Management in the Cloud,Arizona State University,https://www.coursera.org/account/accomplishmen...,https://www.coursera.org/account/accomplishmen...,Data Management in the Cloud.pdf
8,2017-07-02,SHAURYA KHURANA,Data Visualization with Advanced Excel,PwC,https://www.coursera.org/account/accomplishmen...,None,Data Visualization with Advanced Excel.pdf
9,2016-09-27,SHAURYA KHURANA,Data-driven Decision Making,PwC,https://www.coursera.org/account/accomplishmen...,None,Data-driven Decision Making.pdf


In [365]:
# data.to_csv('Certificates.csv',index=False)